<a href="https://colab.research.google.com/github/ashwinamrutphale/aml-homeworks/blob/master/homework-5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.image as mpimg
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import euclidean_distances
import pandas as pd
import scipy as sp
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
activities = ['brush_teeth','drink_glass','climb_stairs','pour_water',
          'use_telephone','eat_meat','walk','comb_hair','eat_soup',
          'descend_stairs','getup_bed','sitdown_chair','liedown_bed','standup_chair']

def compute_kmeans(k,df_input):
  return KMeans(n_clusters=k, random_state=0).fit(df_input)

def compute_feature_vector(df_input,kmeans):
  return np.unique(kmeans.predict(df_input),return_counts=True)[1]

def train_and_predict(train,test,k):
  clf = RandomForestClassifier(n_estimator=10,max_depth=4,random_state=0)
  clf.fit(train[:,:k],train[:,-1])
  return clf.predict(test[:,:k])

def loadall_and_slice(w):
  #TODO:ogic to load all files and return sliced in 32 x 32
  return df

def label_race(row):
  if row[0] == row[1]:
    return 1
  else:
    return 0

def compute_acc(df_original,df_predict):
  d = df_original.join(df_predict)
  d2 = d.apply (lambda row: label_race(row),axis=1)
  return d2[[1]].sum()/len(d2)
  
def consufion_matrix(df_original,df_predict):
  return pd.crosstab(df_original['Activities'], df_predict, rownames=['Actual Activities'], colnames=['Predicted Activities'])

  
   
def main_compute(k,w):
  total_df = loadall_and_slice(w)
  kmeans = compute_kmeans(k,total_df)
  #TODO: for each file - doing it for only 1 - need 32X32 sample here as well
  feature_vector = compute_feature_vector(df,kmeans)
  #2 - is index of climb_stairs in activies array - need code to dynamically index this.
  final_features = np.append(feature_vector,2)
  #TODO:
  #for each label(eg brush_teeth)
  train,test = train_test_split(final_features,test_size=0.33, random_state=10)
  predict  =  train_and_predict(train,test,k) 
  

  
  
  
  

  
  



In [0]:
df = pd.read_csv('/content/gdrive/''My Drive''/aml-homeworks/homework5/Accelerometer-2011-03-24-10-24-39-climb_stairs-f1.txt',delim_whitespace=True)


In [0]:
f = compute_feature_vector(df,compute_kmeans(5,df))
f
#np.unique(f, return_counts=True)[1]

array([56, 39, 45, 54, 75])

In [0]:
len(activities)

14